In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "anything"
from ragas.testset.generator import TestsetGenerator,RunConfig
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.integrations.llama_index import evaluate 
import pandas as pd
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex,Document,Settings
from llama_index.core import VectorStoreIndex,Document,Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core import Document



# generator with openai models
model_name = "gpt-4o"
base_url = "http://localhost:8080/"
max_tokens = 1024*8
generator_llm = ChatOpenAI(model=model_name, base_url=base_url,max_tokens=max_tokens)
critic_llm = ChatOpenAI(model=model_name, base_url=base_url,max_tokens=max_tokens)
embeddings = OpenAIEmbeddings(model='text', base_url=base_url)
runconfig = RunConfig(max_workers=1,max_retries=1)

unit_text_path = '/home/cip/ce/ix05ogym/Majid/LLM/GraphRag/ragtest/output/create_base_text_units.parquet'

texts = pd.read_parquet(unit_text_path)

texts

In [ ]:


Settings.embed_model = embeddings #OpenAIEmbedding(model='text',api_base=base_url,max_retries=1)
Settings.llm=OpenAI(model=model_name,api_base=base_url,max_tokens=max_tokens,max_retries=1)
Settings.chunk_size=1200

# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db")
# create collection
chroma_collection = db.get_or_create_collection("quickstart")


vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store( vector_store )
retriever = index.as_retriever()
rag_pipline = index.as_query_engine(similarity_top_k=2)

rag_pipline.query("hello")


In [ ]:
#add new datqa
#d = texts['chunk'].tolist()
#e = embeddings.embed_documents(d)
#chroma_collection.add(ids=texts['id'].tolist(),documents=d,embeddings=e)
#chroma_collection.count()

In [ ]:
data = chroma_collection.get(include=['documents','embeddings'])
docs = data['documents']
docs_llama = list(map(lambda x:Document(text=x ),docs))
docs_llama

In [ ]:
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
import nest_asyncio
nest_asyncio.apply()
dataset_generator = RagDatasetGenerator.from_documents(
    documents=docs_llama[:20],
    num_questions_per_chunk=1,
    #text_question_template=DEFAULT_QUESTION_GENERATION_PROMPT,
    #question_gen_query=DEFAULT_TEXT_QA_PROMPT,
    show_progress=True,
    workers=1
    
)

len(dataset_generator.nodes) # 1314

rag_dataset =  dataset_generator.generate_dataset_from_nodes()
df = rag_dataset.to_pandas()
df

In [ ]:
from ragchecker.integrations.llama_index import response_to_rag_results


rag_result_list = []
for i,x in df.iterrows():
    response_object = rag_pipline.query(x['query'])
    rag_result = response_to_rag_results(
    query=x['query'],
    gt_answer=x['reference_answer'],
    response_object=response_object,
    )
    rag_result_list.append(rag_result)

rag_results = RAGResults.from_dict({"results": rag_result_list})
print(rag_results)

In [ ]:
from ragchecker import RAGResults, RAGChecker
from ragchecker.metrics import all_metrics



# Create RAGResults object

# initialize ragresults from json/dict

# set-up the evaluator
evaluator = RAGChecker(
    extractor_name=model_name,
    checker_name=model_name,
    batch_size_extractor=1,
    batch_size_checker=1,
    #custom_llm_api_func=generator_llm.invoke,
    checker_api_base=base_url,
    extractor_api_base=base_url,
    
    
)

# evaluate results with selected metrics or certain groups, e.g., retriever_metrics, generator_metrics, all_metrics
evaluator.evaluate(rag_results, all_metrics)
print(rag_results)